# Calculating surface latent heat flux anomalies 

## Importing libraries 

In [1]:
from cartopy import config
import cartopy
import cartopy.crs as ccrs
import cmocean 
import dask as da
from dask.distributed import LocalCluster, Client
from datetime import date
import glob
import holoviews as hv
from matplotlib import pyplot as plt
import numpy as np
import time
import xarray as xr
%pylab inline
local_dir = "/g/data/e14/cp3790/dask-workers"
cluster = LocalCluster(processes=False, local_dir=local_dir)
client = Client(cluster)
client

Populating the interactive namespace from numpy and matplotlib


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.04/lib/python3.7/site-packages/distributed/worker.py:482: UserWarning: The local_dir keyword has moved to local_directory
  warnings.warn("The local_dir keyword has moved to local_directory")


Client Scheduler: inproc://10.0.64.21/22755/1 Dashboard: http://10.0.64.21:8787/status,Cluster Workers: 1 Cores: 8 Memory: 33.67 GB


## Defining functions 

In [2]:
# Function created by Guillaume Serazin

def reshape_data(da):
        da_groupby = list(da.groupby('time.dayofyear'))
        dayofyear = []
        da_dayofyear = []
        for item in list(da_groupby):
            dayofyear.append(item[0])
            da_tmp = item[1]
            da_tmp['time'] = da_tmp['time.year']
            da_tmp = da_tmp.rename({'time': 'year'})
            da_tmp = da_tmp.assign_coords(dayofyear=item[0])
            da_dayofyear.append(da_tmp)
        da_reshaped = xr.concat(da_dayofyear, dim='dayofyear')
        return da_reshaped

## Opening files 

In [4]:
files = sorted(glob.glob('/g/data/e14/cp3790/Charuni/ERA5-SLHF/era5_dailyslhf_*.nc'))

era5_dailyslhf_aus = xr.open_mfdataset(files, combine='by_coords').sel(time=slice('1983', '2012'), longitude=slice(90, 180), latitude=slice(0, -60))
slhf = era5_dailyslhf_aus.slhf
slhf

,Array,Chunk
Bytes,3.80 GB,127.02 MB
Shape,"(10958, 241, 360)","(366, 241, 360)"
Count,141 Tasks,30 Chunks
Type,float32,numpy.ndarray


## Calculations 

In [7]:
# We get the data ready to have a rolling mean performed upon it. This is performed in the reshape_data function (created by Guillaume Serazin)

reshaped_slhf = reshape_data(slhf)
reshaped_slhf

,Array,Chunk
Bytes,3.81 GB,347.04 kB
Shape,"(366, 30, 241, 360)","(1, 1, 241, 360)"
Count,33157 Tasks,10980 Chunks
Type,float32,numpy.ndarray


In [8]:
start = reshaped_slhf[:31] # the first 31 days 
start['dayofyear'] = range(366,397) # the first 31 days will be 'stitched' to the last 31 days 
end = reshaped_slhf[-31:] # the last 31 days 
end['dayofyear'] = range(-30, 1) # the last 31 days will be 'stitched' to the first 31 days 
circular_slhf = xr.concat([end, reshaped_slhf, start], dim = 'dayofyear').chunk({'dayofyear' : 31})
circular_slhf

,Array,Chunk
Bytes,4.46 GB,10.76 MB
Shape,"(428, 30, 241, 360)","(31, 1, 241, 360)"
Count,48277 Tasks,420 Chunks
Type,float32,numpy.ndarray


In [9]:
raw_slhf = circular_slhf.mean('year')
slhf_climatology_smooth = raw_slhf.rolling(dayofyear = 15, center = True).mean() # smoothen it once with a 15-day rolling window
slhf_climatology_smoother = slhf_climatology_smooth.rolling(dayofyear = 31, center = True).mean() # smoothen it again by running a 31-day window 
slhf_climatology = slhf_climatology_smoother.isel(dayofyear = slice(31,-31)) # drop the first and last 31 days 

In [10]:
slhf_climatology

<xarray.DataArray (dayofyear: 366, latitude: 241, longitude: 360)>
dask.array<getitem, shape=(366, 241, 360), dtype=float32, chunksize=(31, 241, 360), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 0.0 -0.25 -0.5 -0.75 ... -59.5 -59.75 -60.0
  * longitude  (longitude) float32 90.0 90.25 90.5 90.75 ... 179.25 179.5 179.75
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 360 361 362 363 364 365 366

In [11]:
xr.Dataset({'climatology': slhf_climatology}).to_netcdf('/g/data/e14/cp3790/Charuni/Heatwaves/slhf-clim-aus.nc',
                                              encoding={'climatology': 
                                                        {'chunksizes': (100, slhf_climatology.shape[1], slhf_climatology.shape[2]),
                                                         'zlib': True,
                                                         'shuffle': True, 
                                                         'complevel': 2}}) 

distributed.worker - WARNING -  Compute Failed
Function:  getter
args:      (ImplicitToExplicitIndexingAdapter(array=CopyOnWriteArray(array=LazilyOuterIndexedArray(array=<xarray.backends.netCDF4_.NetCDF4ArrayWrapper object at 0x7fd290699370>, key=BasicIndexer((slice(None, None, None), slice(None, None, None), slice(None, None, None)))))), (slice(0, 366, None), slice(360, 601, None), slice(1080, 1440, None)))
kwargs:    {}
Exception: RuntimeError('NetCDF: HDF error')



RuntimeError: NetCDF: HDF error